<a href="https://colab.research.google.com/github/sisifo3/P_T_3/blob/main/Densenet121_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#all the classes and 180000 img

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/NIH_dataset/CheXpert-v1.0-small-1.zip 

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: CheXpert-v1.0-small/train/patient21643/study3/view2_lateral.jpg  
  inflating: CheXpert-v1.0-small/train/patient21643/study3/view1_frontal.jpg  
   creating: CheXpert-v1.0-small/train/patient21643/study1/
  inflating: CheXpert-v1.0-small/train/patient21643/study1/view1_frontal.jpg  
   creating: CheXpert-v1.0-small/train/patient21643/study2/
  inflating: CheXpert-v1.0-small/train/patient21643/study2/view1_frontal.jpg  
   creating: CheXpert-v1.0-small/train/patient63218/
   creating: CheXpert-v1.0-small/train/patient63218/study1/
  inflating: CheXpert-v1.0-small/train/patient63218/study1/view1_frontal.jpg  
   creating: CheXpert-v1.0-small/train/patient08046/
   creating: CheXpert-v1.0-small/train/patient08046/study1/
  inflating: CheXpert-v1.0-small/train/patient08046/study1/view1_frontal.jpg  
   creating: CheXpert-v1.0-small/train/patient08046/study5/
  inflating: CheXpert-v1.0-small/train/patient08046/s

In [3]:
import cv2
import numpy as np
from PIL import Image
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates

random_state = np.random.RandomState(0)

def clahe(image_name):
    image = cv2.imread(image_name, 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    he_img = clahe.apply(image)
    image = cv2.cvtColor(he_img, cv2.COLOR_GRAY2RGB)

    return Image.fromarray(image)

def elastic_transform(image, alpha, sigma):
    image = np.asarray(image)

    if len(image.shape) < 3:
        image = image.reshape(image.shape[0], image.shape[1], -1)

    shape = image.shape

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]), indexing='ij')
    indices = np.reshape(x + dx, (-1, 1)), np.reshape(y + dy, (-1, 1)), np.reshape(z, (-1, 1))

    distorted_image = map_coordinates(image, indices, order=0, mode='reflect')

    return Image.fromarray(distorted_image.reshape(image.shape))

def get_aug(image, index):
    if (index == 0):
        return image
    else:
        alpha = np.random.uniform(500,1000)
        sigma = np.random.uniform(9, 10)
        input_transformed = elastic_transform(image, alpha=alpha, sigma=sigma)
        return  input_transformed

In [4]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

from torchvision import transforms

import torch
from torch.utils.data import Dataset

def getImagesLabels(filename, policy):
    df = pd.read_csv(filename)
    relevant_cols = ['Path', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly' ,
                    'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                    'Pneumonia', 'Atelectasis' ,'Pneumothorax', 'Pleural Effusion',
                    'Pleural Other', 'Fracture', 'Support Devices']

    df = df[relevant_cols]
    df = df.replace(np.nan, 0.0)

    df = df.replace(-1.0, 0.0)


    X = df['Path']
    y = df[['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly' ,
                    'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                    'Pneumonia', 'Atelectasis' ,'Pneumothorax', 'Pleural Effusion',
                    'Pleural Other', 'Fracture', 'Support Devices']]
   
    return np.asarray(X), np.asarray(y)


class CheXpertDataset(Dataset):
    def __init__(self, image_list, labels, transform=None, test=False):
        """
        image_list: list of paths containing images
        labels: corresponding multi-class labels of image_list
        transform: optional transform to be applied on a sample.
        """
        self.image_names = image_list
        self.gt = labels
        if (len(np.unique(labels)) > 2):
            self.labels = multihot(labels)
        else:
            self.labels = labels

        self.transform = transform
        self.test = test

    def __getitem__(self, index):
        """Take the index of item and returns the image and its labels"""

        image_name = self.image_names[index]
        # image = Image.open(image_name).convert('RGB')
        image = clahe(image_name)

        label = self.labels[index]
        gt = self.gt[index]

        # index = random.choice([0,1,2,3])
        # if (self.test==False):
        #     image = get_aug(image, index)

        # plt.imshow(image)
        # plt.show()

        if self.transform is not None:
            image = self.transform(image)
        else:
            self.transform = transform = transforms.Compose([
                    transforms.Resize((320, 320)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                    ])
            image = self.transform(image)
        if self.test:
            return image, torch.FloatTensor(gt)
        else:
            return image, torch.FloatTensor(label)

    def __len__(self):
        return len(self.image_names)

In [5]:
import torch
import  torchvision
import torch.nn as nn
from torchsummary import summary

class DenseNet121(nn.Module):
    """Model modified.
    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.
    """
    def __init__(self, out_size, test=False):
        super(DenseNet121, self).__init__()
        self.test = test
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, num_ftrs),
            nn.BatchNorm1d(num_ftrs),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_ftrs//2),
            nn.BatchNorm1d(num_ftrs//2),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(num_ftrs//2, out_size),
        )

    def forward(self, x):
        x = self.densenet121(x)

        if self.test:
            return torch.sigmoid(x)
        else:
            return x


In [6]:
import torch
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import os

class Trainer():
    def __init__(self, model, train_loader, val_loader):
        """ Initialze the model and dataloaders for the trainer
        Arguments:
            model : the model to be trained
            train_loader : DataLoader containing training data
            val_loader : DataLoader containing validation data
        """
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader

        self.losses = {'train':[], 'validation':[]}

    def get_device(self):
        """ Function to check for GPU if available, if not return CPU as device
        Returns:
            device : gpu or cpu detected by torch
        """
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        return self.device

    def compile(self, lr, loss_fn, scheduler=False):
        """ Function to initialize loss, optimizer and scheduler for the model to be trained
        Arguments:
            lr : learning rate of the model
            loss : the loss function to be minimized, could be from torch.nn or could be user-defined
        Keyword Arguments:
            scheduler : whether to use scheduler if loss plateaus (default: False)
        """
        self.learning_rate = lr
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
        self.loss_fn = loss_fn

        if (scheduler):
            self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-6)
            # self.scheduler = StepLR(self.optimizer, step_size=5, gamma=0.1)

    def load_checkpoint(self, log_path, model_path):
        """ Function to load checkpoints from a previously saved model
        Arguments:
            log_path : path to saved log file
            model_path : path to saved model
        Returns:
            start_epoch : the epoch from which to resume the training
            model : the model with loaded weights
            optimizer : the optimizer with the previous state
            scheduler : the scheduler with the previous state
            losses : the dictionary populated with loss values from previous epochs
        """
        # Note: Input model & optimizer should be pre-defined. This routine only updates their states
        start_epoch = 0

        self.log_path = log_path
        self.model_path = model_path

        print(self.log("\nLoading checkpoint from '{}'\n".format(model_path)))

        checkpoint = torch.load(model_path)
        start_epoch = checkpoint['epoch'] + 1
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        try:
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        except:
            pass

        self.losses = checkpoint['loss']
        val_min = min(self.losses['validation'])

        print(self.log("--> Loaded checkpoint from '{}'\nResuming training from epoch {}\n\n"
                    .format(model_path, start_epoch)))

        return start_epoch, self.model, self.optimizer, self.scheduler, self.losses, val_min
        # return start_epoch, self.model, self.optimizer, self.losses, val_min


    def epoch_train(self, print_every=50):
        model = self.model
        loss_fn = self.loss_fn
        optimizer = self.optimizer
        device = self.device

        train_loss = 0.0
        model.train()
        for batch_idx, (data, target) in enumerate(self.train_loader):

            data, target = data.float().to(device), target.float().to(device)

            optimizer.zero_grad()

            pred = model(data)
            loss = loss_fn(pred, target)

            train_loss += ((1 / (batch_idx + 1)) * (loss.item()/data.size(0) - train_loss))
            loss.backward()
            optimizer.step()

            if (batch_idx % print_every == 0):
                print('Epoch {}\tBatch [{}/{}]\t\tTraining Loss: {}'.format(self.epoch+1, batch_idx+1, len(self.train_loader), train_loss))

        return train_loss

    def epoch_val(self):
        model = self.model
        loss_fn = self.loss_fn
        device = self.device

        valid_loss = 0.0
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(self.val_loader):
                data, target = data.float().to(device), target.float().to(device)

                val_pred = model(data)
                val_loss = loss_fn(val_pred, target)

                valid_loss += ((1 / (batch_idx + 1)) * (val_loss.item()/data.size(0) - valid_loss))

        return valid_loss

    def train(self, n_epochs, batch_size, log_path=None, model_path=None):
        """ Function to the train the model
        Arguments:
            n_epochs : number of epochs for which the model should be trained
            batch_size : batch_size of the DataLoaders
            log_path : path to saved log file
            model_path : path to saved model
        Returns:
            model : trained model
            losses : the dictionary containing the loss values
        """
        start = datetime.now()

        model = self.model
        optimizer = self.optimizer
        try:
            scheduler = self.scheduler
        except:
            pass
        losses = self.losses
        valid_loss_min = np.Inf
        start_epoch = 0

        if (log_path is None and model_path is None):
            self.log_path = str(start.strftime('%d-%m-%Y-%H:%M:%S')+'_train_log')
            self.model_path = '{}_model.pt'.format(start.strftime('%d-%m-%Y-%H:%M:%S'))
            self.log('Learning rate: {}, Batch size: {}\n\n'.format(self.learning_rate, batch_size))

        elif (os.path.exists(model_path) and os.path.exists(log_path)):
            # start_epoch, model, optimizer, scheduler, losses = self.load_checkpoint(log_path, model_path)
            start_epoch, model, optimizer, scheduler, losses, valid_loss_min = self.load_checkpoint(log_path, model_path)
            valid_loss_min = min(losses['validation'])

        else:
            print('[!] Specified model path or log path does not exist.')
            return

        # loss_fn = self.loss_fn

        checkpoint = {}

        for self.epoch in range(start_epoch, start_epoch+n_epochs):

            # self.epoch = epoch

            train_loss = self.epoch_train(print_every=500)
            valid_loss = self.epoch_val()

            print(self.log('\nEpoch: [{}/{}] \tTraining Loss: {:.5f} \tValidation Loss: {:.5f}\n'.format(
                                                    self.epoch+1, start_epoch+n_epochs, train_loss, valid_loss)))
            print('-'*100)

            #####----CHECKPOINTING----#####
            if (valid_loss < valid_loss_min):
                print(self.log("Saving model.  Validation loss:... {:.5f} --> {:.5f}\n".format(valid_loss_min, valid_loss)))
                print('*'*100)
                valid_loss_min = valid_loss

                checkpoint['model_state_dict'] = model.state_dict()
                checkpoint['optimizer_state_dict'] = optimizer.state_dict()
                try:
                    checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                except:
                    pass
                print()

            try:
                scheduler.step(valid_loss)
            except:
                pass

            losses['train'].append(train_loss)
            losses['validation'].append(valid_loss)

            checkpoint['epoch'] = self.epoch
            checkpoint['loss'] = losses
            torch.save(checkpoint, self.model_path)

            self.draw_loss_curve('{}_losses.png'.format(self.model_path.split('_')[0]))

        end = datetime.now()
        time = str(end - start).split('.')[0]
        print(self.log("\nCompleted training in {}\n".format(time)))

        return model, losses

    def log(self, info):
        """ Function to create and update the log file
        Arguments:
            info : the update information to write on the log file
        Returns:
            info : the logged information to be printed while training
        """
        log_path = self.log_path

        if not os.path.exists(log_path):
            file = open(log_path, 'w')
            file.write(info)
            file.close()
        else:
            file = open(log_path, 'a')
            file.write(info)
            file.close()

        return info.strip('\n')

    def draw_loss_curve(self, fpath, losses=None):
        """ Function to generate loss curve for the training process
        Arguments:
            fpath : the filepath to save the loss curve in
        """
        if losses is None:
            losses = self.losses
        # plt.ylim([0,2])
        plt.plot(losses['train'], label='Training loss')
        plt.plot(losses['validation'], label='Validation loss')
        plt.legend()
        plt.savefig(fpath)
        # plt.show()
        plt.close()

In [ ]:
import sys

import numpy as np
from torchvision import transforms

import torch
import torch.nn as nn
#from dataloader import CheXpertDataset, getImagesLabels
#from network import DenseNet121
from torch.utils.data import DataLoader
#from trainer import Trainer

# batchSize = int(sys.argv[1])
# lr = float(sys.argv[2])
# epochs = int(sys.argv[3])
# num_classes = int(sys.argv[4])
# policy = sys.argv[5]
policy = 'both'

train_csv = 'CheXpert-v1.0-small/train.csv'
# train_csv = 'train.csv'

#log_path = '/content/drive/MyDrive/weightsDensenet121_100,000/15-10-2021-18:30:37_train_log'
#model_path = '/content/drive/MyDrive/weightsDensenet121_100,000/15-10-2021-18:30:37_model.pt'
#log_path = '/content/drive/MyDrive/weightsDensenet121_100,000/Copia de Copia_4 de 15-10-2021-18:30:37_train_log'
#model_path = '/content/drive/MyDrive/weightsDensenet121_100,000/Copia de Copia_4 de 15-10-2021-18:30:37_model.pt'
#log_path = '/content/drive/MyDrive/wieghtsDensenet121_180000_14classes/18-10-2021-18:26:12_train_log'
#model_path = '/content/drive/MyDrive/wieghtsDensenet121_180000_14classes/18-10-2021-18:26:12_model.pt'
log_path = '/content/drive/MyDrive/wieghtsDensenet121_180000_14classes/Copia de Copia_6 de 18-10-2021-18:26:12_train_log'
model_path = '/content/drive/MyDrive/wieghtsDensenet121_180000_14classes/Copia de Copia_6 de 18-10-2021-18:26:12_model.pt'

cropSize = 320

transform = transforms.Compose([
                    transforms.RandomResizedCrop(cropSize),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(10),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                    ])

train_images, train_labels = getImagesLabels(train_csv, policy)
# train_images, train_labels = getImagesLabels(train_csv)

val_images, val_labels = train_images[180001:], train_labels[180001:]
train_images, train_labels = train_images[:180000], train_labels[:180000]

#val_images, val_labels = train_images[180001:], train_labels[180001:]
#train_images, train_labels = train_images[:180000], train_labels[:180000]

# val_images, val_labels = train_images[270000:], train_labels[270000:]
# train_images, train_labels = train_images[:270000], train_labels[:270000]

# val_images, val_labels = train_images[8000:10000], train_labels[8000:10000]
# train_images, train_labels = train_images[:8000], train_labels[:8000]

print('No. of images:\n\t|--Training: {}\n\t|--Validation: {}\n'.format(len(train_images), len(val_images)))

train_dataset = CheXpertDataset(train_images, train_labels, transform)
val_dataset = CheXpertDataset(val_images, val_labels, transform)

### hyperparameters ###
batchSize = 32
lr = 0.0001
epochs = 3
num_classes = 14

### data loaders ###
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True,  num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False,  num_workers=2, pin_memory=True)

### initializing the model ###
model = DenseNet121(num_classes)
model = torch.nn.DataParallel(model)
model.cuda()

### training ###
trainer = Trainer(model, train_loader, val_loader)
device = trainer.get_device()

# moving the model to device
model.to(device)

# defining the losses with class weights


pos_weights = [8.982306420624637, 19.690313020929803, 7.274592592592593, 1.1160436063306844, 23.321140866536034
               , 3.2761933927956206, 14.112899952648313, 35.99519788044378, 5.693851869606903, 10.487762237762238
               , 1.5922006799169248, 62.41583877377235, 23.713938053097344, 0.925966155464177]


pos_weights = torch.FloatTensor(pos_weights).cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

trainer.compile(lr, criterion, scheduler=True)

trainer.train(epochs, batchSize, log_path, model_path)

#trainer.train(epochs, batchSize)

No. of images:
	|--Training: 180000
	|--Validation: 43413



Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

Loading checkpoint from '/content/drive/MyDrive/wieghtsDensenet121_180000_14classes/Copia de Copia_6 de 18-10-2021-18:26:12_model.pt'
--> Loaded checkpoint from '/content/drive/MyDrive/wieghtsDensenet121_180000_14classes/Copia de Copia_6 de 18-10-2021-18:26:12_model.pt'
Resuming training from epoch 15


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 16	Batch [1/5625]		Training Loss: 0.02596176601946354
Epoch 16	Batch [501/5625]		Training Loss: 0.02907475968915782
Epoch 16	Batch [1001/5625]		Training Loss: 0.028930875274550884
Epoch 16	Batch [1501/5625]		Training Loss: 0.029044964415909112
Epoch 16	Batch [2001/5625]		Training Loss: 0.029166504226881874
Epoch 16	Batch [2501/5625]		Training Loss: 0.029202110283389555
Epoch 16	Batch [3001/5625]		Training Loss: 0.02925309793752396
Epoch 16	Batch [3501/5625]		Training Loss: 0.029216093562119856
Epoch 16	Batch [4001/5625]		Training Loss: 0.02921593555204347
Epoch 16	Batch [4501/5625]		Training Loss: 0.029247084361871612
Epoch 16	Batch [5001/5625]		Training Loss: 0.02926479069073957
Epoch 16	Batch [5501/5625]		Training Loss: 0.02924415437645845
Epoch: [16/18] 	Training Loss: 0.02925 	Validation Loss: 0.02921
----------------------------------------------------------------------------------------------------
Epoch 17	Batch [1/5625]		Training Loss: 0.025015275925397873
Epoch 17	Batch 